##### Investment Strategy


In [1]:
import pandas as pd
import numpy as np
import time
import statsmodels.api as sm
from tqdm import tqdm
import os

# get the absolute path of the notebook file
notebook_path = os.path.abspath("InvestmentStrategy_Dividend_Yield.ipynb")
# get the directory containing the notebook file
path_data = os.path.dirname(notebook_path)

In [2]:
os.listdir(path_data)

['.ipynb_checkpoints',
 'crsp_div.csv',
 'CRSP_Dividend.ipynb',
 'dividend_yearly.csv',
 'F-F_Research_Data_Factors_2018.csv',
 'InvestmentStrategy_Dividend_Yield.ipynb']

## Step 1: Preparing the CRSP file

In [3]:
print("Prepare CRSP file")
t = time.time() # record the current time, so we can measure how long the code takes to run

crsp = pd.read_csv('crsp_div.csv')
print('Completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block

Prepare CRSP file
Completed in 2.2s


In [4]:
crsp.head(10)

,PERMNO,date,SHRCD,EXCHCD,TICKER,COMNAM,PERMCO,PRC,RET,SHROUT,RETX
0,10000,1985-12-31,NaN,NaN,NaN,NaN,7952,NaN,NaN,NaN,NaN
1,10000,1986-01-31,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-4.37500,C,3680.0,C
2,10000,1986-02-28,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-3.25000,-0.257143,3680.0,-0.257143
3,10000,1986-03-31,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-4.43750,0.365385,3680.0,0.365385
4,10000,1986-04-30,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-4.00000,-0.098592,3793.0,-0.098592
5,10000,1986-05-30,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-3.10938,-0.222656,3793.0,-0.222656
6,10000,1986-06-30,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-3.09375,-0.005025,3793.0,-0.005025
7,10000,1986-07-31,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-2.84375,-0.080808,3793.0,-0.080808
8,10000,1986-08-29,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-1.09375,-0.615385,3793.0,-0.615385
9,10000,1986-09-30,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-1.03125,-0.057143,3793.0,-0.057143


In [5]:
sum(crsp['RET'].isna())

34046

In [6]:
crsp.dtypes

PERMNO      int64
date       object
SHRCD     float64
EXCHCD    float64
TICKER     object
COMNAM     object
PERMCO      int64
PRC       float64
RET        object
SHROUT    float64
RETX       object
dtype: object

#### Formatting Data:

In [7]:
### formatting ###
# make all variable names lowercase
t = time.time()
crsp.columns = map(str.lower,crsp.columns)

# Changes the returns to number format. Non-numeric data will be NAN
crsp['ret'] = pd.to_numeric(crsp['ret'],errors='coerce') 

# Change the dateformat
crsp['date'] = pd.to_datetime(crsp['date'], format='%Y-%m-%d')

# Create separate 'year' and 'month' variables (we will use them later to merge CRSP with Compustat)
crsp['year'] = crsp['date'].apply(lambda date: date.year)
crsp['month'] = crsp['date'].apply(lambda date: date.month)

# Calculate market cap
crsp['mktcap'] = crsp['shrout'] * crsp['prc'].abs()


### Some basic data cleaning ###
# keep only common shares
crsp = crsp[crsp['shrcd'].isin([10,11])]

# keep only stocks from NYSE, AMEX and NASDAQ
crsp = crsp[crsp['exchcd'].isin([1,2,3])]

#    Drop the duplicates.
crsp = crsp.drop_duplicates(subset=['date','permno'])

print('Completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block

Completed in 12.2s


### Step 2: Loading the Dividened_Yield File (Already Prepared in another notebook)

In [8]:
print("Prepare dividend_yield file")
t = time.time() 
dv = pd.read_csv('dividend_yearly.csv')
print('Completed in %.1fs' % (time.time()-t))

Prepare dividend_yield file
Completed in 0.5s


In [9]:
len(dv)

140899

In [10]:
dv=dv.drop('Unnamed: 0',axis=1)
dv.head()

,PERMNO,SHRCD,EXCHCD,TICKER,COMNAM,PERMCO,PRC,SHROUT,date,MARKET_CAP,DVY_M,DVY_A
0,10000,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-3.09375,3793.0,1986-06-30,11734.59375,0.000000,0.000000
1,10000,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,NaN,3893.0,1987-06-30,NaN,0.000000,NaN
2,10001,11.0,3.0,GFGC,GREAT FALLS GAS CO,7953,-6.12500,985.0,1986-06-30,6033.12500,194.865751,0.032299
3,10001,11.0,3.0,GFGC,GREAT FALLS GAS CO,7953,5.87500,991.0,1987-06-30,5822.12500,419.734086,0.072093
4,10001,11.0,3.0,GFGC,GREAT FALLS GAS CO,7953,6.25000,992.0,1988-06-30,6200.00000,404.015932,0.065164


In [11]:
sum(dv['DVY_A'].isna())

2378

We get rid of the NaN values for dividend yield.

In [12]:
dv=dv[~dv['DVY_A'].isna()]

In [13]:
sum(dv['DVY_A']==0)/len(dv)

0.5623335089986355

56% of the rows have 0 dividend yield. We will put them into a separate portfolio and leave those with positive dividend yield to build five portfolios.

In [14]:
dv_0=dv[dv['DVY_A']==0].reset_index(drop=True)

In [15]:
dv=dv[dv['DVY_A']>0].reset_index(drop=True)

#### Formatting:

In [16]:
#For dv dataframe
t = time.time() 

dv.columns = map(str.lower,dv.columns)

# Change the dateformat 
dv['date'] = pd.to_datetime(dv['date'])

# Create separate 'year' and 'month' variables
dv['year'] = dv['date'].apply(lambda x: x.year)
dv['month'] = dv['date'].apply(lambda x: x.month)

print('Completed in %.1fs' % (time.time()-t))

Completed in 0.2s


In [17]:
#For dv_0 dataframe
t = time.time() 

dv_0.columns = map(str.lower,dv_0.columns)

# Change the dateformat 
dv_0['date'] = pd.to_datetime(dv_0['date'])

# Create separate 'year' and 'month' variables
dv_0['year'] = dv_0['date'].apply(lambda x: x.year)
dv_0['month'] = dv_0['date'].apply(lambda x: x.month)

print('Completed in %.1fs' % (time.time()-t))

Completed in 0.3s


### Step 3: Sort stocks into portfolios and calculate returns

In [18]:
len(dv['permno'].unique())

9421

In [19]:
dv_0.head()

,permno,shrcd,exchcd,ticker,comnam,permco,prc,shrout,date,market_cap,dvy_m,dvy_a,year,month
0,10000,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-3.09375,3793.0,1986-06-30,11734.59375,0.0,0.0,1986,6
1,10001,11.0,3.0,EWST,ENERGY WEST INC,7953,6.66000,2599.0,2004-06-30,17309.34000,0.0,0.0,2004,6
2,10001,11.0,3.0,EWST,ENERGY WEST INC,7953,9.05000,2913.0,2005-06-30,26362.65000,0.0,0.0,2005,6
3,10002,10.0,3.0,MBNC,MOBILE NATIONAL CORP,7954,-12.68750,1175.0,1986-06-30,14907.81250,0.0,0.0,1986,6
4,10002,10.0,3.0,MBNC,MOBILE NATIONAL CORP,7954,-13.50000,1175.0,1987-06-30,15862.50000,0.0,0.0,1987,6


## Univariate sorting by dividend yield

In [20]:
#Create 5 portfolios based on dv dataframe (companies with positive dividend yield)
print("Create portfolios")
t = time.time() # reset our timer

# loop over all years in the data
portfolios = [] # create an empty list to collect the portfolio returns
for year in tqdm(range(1985,2017),desc="years"):
    # take the companies that were alive at t-1
    permno_list=list(dv[dv['year']==year-1]['permno'].unique()) 
    
    # get the sorting variable for these companies at t-1
    sorting_data = dv.loc[(dv['year']==(year-1)) & \
                           (dv['permno'].isin(permno_list)), \
                           ['permno','permco','dvy_a']]
    
    if(len(sorting_data)==0):
        continue
    
    # sort into 5 baskets by annual dividend yield
    nportfolios = 4 # number of portfolios
    sorting_data['rank'] = pd.qcut(sorting_data['dvy_a'],nportfolios, labels=False)#, precision=4)
    
    # select the return data with some time lag to make sure that the accounting information is public (data from July at year t to June in year t+1)
    crsp_window = crsp[((crsp['year']==year) & (crsp['month']>=6)) | \
                       ((crsp['year']==year+1) & (crsp['month']<=6))]
    
    # create the portfolio returns for the current window and collect them in portfolios_window
    portfolios_window = [] 
    for p in range(nportfolios):
        # get list of permnos that are in this portfolio
        basket = sorting_data.loc[sorting_data['rank'] == p,'permno'].tolist()
        
        # get returns of these permnos
        crsp_p_firms = crsp_window[crsp_window['permno'].isin(basket)]
        
        # pivot returns
        returns = crsp_p_firms.pivot(index='date', columns='permno', values='ret')
        returns = returns.iloc[1:,:] # drop the first row
        
        # create equally weighted portfolio (monthly rebalancing)
        return_port = returns.mean(axis=1)
        return_port.name = str(p)
        
        # collect portfolio returns in dec_port
        portfolios_window += [return_port]
        
    # merge the portfolios
    portfolios_window = pd.concat(portfolios_window,axis=1)
        
    # collect results in portfolios
    portfolios += [portfolios_window]

# merge the returns from all windows
portfolios = pd.concat(portfolios,axis=0)

print('Step 3 completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block

Create portfolios


years:   0%|                                                                                    | 0/32 [00:00<?, ?it/s]C:\Users\hongt\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
years:   9%|███████▏                                                                    | 3/32 [00:01<00:18,  1.54it/s]C:\Users\hongt\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
years:  12%|█████████▌                                                                  | 4/32 [00:02<00:13,  2.10it/s]C:\Users\hongt\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
years:  16%|███████████▉                                                                | 5/32 [00:02<00:09,  2.74it/s]C:\Users\hongt\anaconda3\Lib\site-packages\numpy\l

Step 3 completed in 4.8s


In [21]:
#Create long-short portfolio
portfolios['4']=portfolios['3']-portfolios['0']
portfolios

,0,1,2,3,4
date,,,,,
1987-07-31,0.038191,0.040972,0.042870,0.021221,-0.016970
1987-08-31,0.029609,0.032789,0.035770,0.026213,-0.003396
1987-09-30,-0.026768,-0.018685,-0.006135,-0.012750,0.014018
1987-10-30,-0.272076,-0.251540,-0.246018,-0.177002,0.095075
1987-11-30,-0.052665,-0.046193,-0.034254,-0.020470,0.032195
...,...,...,...,...,...
2010-02-26,0.047873,0.035243,0.007620,0.041849,-0.006024
2010-03-31,0.070751,0.083484,0.088512,0.043454,-0.027298
2010-04-30,0.055061,0.076558,0.120142,0.163848,0.108787


In [22]:
#Create 5 portfolios based on dv dataframe (companies with dividend yield = 0)
print("Create portfolios")
t = time.time() # reset our timer

# loop over all years in the data
portfolios_0 = [] # create an empty list to collect the portfolio returns
for year in tqdm(range(1985,2017),desc="years"):
    # take the companies that were alive at t-1
    permno_list=list(dv_0[dv_0['year']==year-1]['permno'].unique()) 
    
    # get the sorting variable for these companies at t-1
    sorting_data = dv.loc[(dv_0['year']==(year-1)) & \
                           (dv_0['permno'].isin(permno_list)), \
                           ['permno','permco','dvy_a']]
    
    if(len(sorting_data)==0):
        continue
    
    # Create 1 portfolio with all companies with annual dividend yield = 0
    nportfolios = 1 # number of portfolios
    sorting_data['rank'] = pd.qcut(sorting_data['dvy_a'],nportfolios, labels=False)#, precision=4)
    
    # select the return data with some time lag to make sure that the accounting information is public (data from July at year t to June in year t+1)
    crsp_window = crsp[((crsp['year']==year) & (crsp['month']>=6)) | \
                       ((crsp['year']==year+1) & (crsp['month']<=6))]
    
    # create the portfolio returns for the current window and collect them in portfolios_window
    portfolios_window = [] 
    for p in range(nportfolios):
        # get list of permnos that are in this portfolio
        basket = sorting_data.loc[sorting_data['rank'] == p,'permno'].tolist()
        
        # get returns of these permnos
        crsp_p_firms = crsp_window[crsp_window['permno'].isin(basket)]
        
        # pivot returns
        returns = crsp_p_firms.pivot(index='date', columns='permno', values='ret')
        returns = returns.iloc[1:,:] # drop the first row
        
        # create equally weighted portfolio (monthly rebalancing)
        return_port = returns.mean(axis=1)
        return_port.name = str(p)
        
        # collect portfolio returns in dec_port
        portfolios_window += [return_port]
        
    # merge the portfolios
    portfolios_window = pd.concat(portfolios_window,axis=1)
        
    # collect results in portfolios
    portfolios_0 += [portfolios_window]

# merge the returns from all windows
portfolios_0 = pd.concat(portfolios_0,axis=0)


print('Step 3 completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block



Create portfolios


years:   0%|                                                                                    | 0/32 [00:00<?, ?it/s]C:\Users\hongt\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
years:   9%|███████▏                                                                    | 3/32 [00:00<00:01, 23.47it/s]C:\Users\hongt\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Users\hongt\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Users\hongt\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
years:  19%|██████████████▎                                                             | 6/32 [00:00<00:01, 13.43it/s]C:\Users\ho

Step 3 completed in 2.0s


In [23]:
#Portfolio number 5 is our zero dividend portfolio
portfolios['5']= portfolios_0['0']
portfolios

,0,1,2,3,4,5
date,,,,,,
1987-07-31,0.038191,0.040972,0.042870,0.021221,-0.016970,0.037230
1987-08-31,0.029609,0.032789,0.035770,0.026213,-0.003396,0.030932
1987-09-30,-0.026768,-0.018685,-0.006135,-0.012750,0.014018,-0.016656
1987-10-30,-0.272076,-0.251540,-0.246018,-0.177002,0.095075,-0.240797
1987-11-30,-0.052665,-0.046193,-0.034254,-0.020470,0.032195,-0.039686
...,...,...,...,...,...,...
2010-02-26,0.047873,0.035243,0.007620,0.041849,-0.006024,0.038760
2010-03-31,0.070751,0.083484,0.088512,0.043454,-0.027298,0.095319
2010-04-30,0.055061,0.076558,0.120142,0.163848,0.108787,0.077118


In [24]:
portfolios.to_excel('portfolios_final.xlsx')

### Step 4: Performance Evaluation
#### Step 4a: Merge Portfolio returns with Fama French data

In [25]:
### load and prepare fama french data ###
# load Fama French monthly factors
ff = pd.read_csv('F-F_Research_Data_Factors_2018.csv')
# rename columns
ff.rename({'Mkt-RF':'ExMkt',
           'DATE':'date'},axis=1,inplace=True)

# date variables
ff['year'] = ff['date'] // 100
ff['month'] = ff['date'] % 100
ff.set_index('date',inplace=True)


### formatting ###
# FF data is in percent. Convert to simple returns
ff[['ExMkt', 'SMB', 'HML', 'RF']] /= 100


### merge portfolio returns with Fama French data ###
# date variables
portfolios_ff = portfolios.copy() # create a copy of the portfolios dataframe so we can use it again later
portfolios_ff['year'] = portfolios_ff.index.year
portfolios_ff['month'] = portfolios_ff.index.month

# merge
portfolios_ff = pd.merge(portfolios_ff,ff,on=['year','month'])

### Step 4b: Regressions

In [26]:
# show average returns (annualized and in percent)
print("Average returns (annualized percent)\n",((1+portfolios.mean(axis=0))**12-1)*100)

# Calculate the excess returns
for p in range(6):
    portfolios_ff['ExRet_'+str(p)] = portfolios_ff[str(p)]-portfolios_ff['RF']

Average returns (annualized percent)
 0    10.740896
1    11.544543
2    11.414925
3    12.492999
4     1.595771
5    12.638745
dtype: float64


In [27]:
### Market model regressions ###
table_capm = []
for p in range(6):
    # regress portfolio excess return on market excess return
    results = sm.OLS(portfolios_ff['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff['ExMkt'])).fit()
    
    # collect results
    table_row = pd.DataFrame({'alpha':results.params['const'],
                              'beta_mkt':results.params['ExMkt'],
                              'alpha_t':results.tvalues['const'],
                              'rmse':np.sqrt(results.mse_resid),
                              'R2':results.rsquared},
                             index=[p])
    
    table_capm += [table_row]

In [28]:
# Combine the results for all portfolios
table_capm = pd.concat(table_capm,axis=0)
table_capm.index.name = 'portfolio'

# show results
print("CAPM\n",table_capm)

CAPM
               alpha  beta_mkt   alpha_t      rmse        R2
portfolio                                                  
0          0.001520  0.888347  0.954061  0.022679  0.754634
1          0.002645  0.746606  1.475068  0.025525  0.631670
2          0.002749  0.691243  1.418261  0.027593  0.557119
3          0.003808  0.623045  1.673264  0.032396  0.425753
4         -0.001491 -0.264915 -0.845319  0.025106  0.182463
5          0.003359  0.776022  2.057416  0.023240  0.690883


In [29]:
### Three Factor model regressions ###
table_ff = []
for p in range(6):
    # regress portfolio excess return on market excess return
    results = sm.OLS(portfolios_ff['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff[['ExMkt','SMB','HML']])).fit()
    
    # collect results
    table_row = pd.DataFrame({'alpha':results.params['const'],
                              'beta_mkt':results.params['ExMkt'],
                              'beta_size':results.params['SMB'],
                              'beta_hml':results.params['HML'],
                              'alpha_t':results.tvalues['const'],
                              'rmse':np.sqrt(results.mse_resid),
                              'R2':results.rsquared},
                             index=[p])
    
    table_ff += [table_row]

# Combine the results for all portfolios
table_ff = pd.concat(table_ff,axis=0)
table_ff.index.name = 'quintile'

# show results
print("Fama-French 3\n",table_ff)

Fama-French 3
              alpha  beta_mkt  beta_size  beta_hml   alpha_t      rmse  \
quintile                                                                
0        -0.000790  0.979954   0.405876  0.599966 -0.782840  0.014218   
1        -0.000118  0.878141   0.366892  0.714892 -0.101198  0.016376   
2        -0.000363  0.842390   0.397482  0.805113 -0.305473  0.016757   
3         0.000421  0.776966   0.489441  0.877368  0.278483  0.021282   
4        -0.002608 -0.201787   0.094682  0.287830 -1.527846  0.024038   
5         0.000707  0.895368   0.389454  0.687045  0.762596  0.013057   

                R2  
quintile            
0         0.904521  
1         0.849895  
2         0.838284  
3         0.754630  
4         0.257959  
5         0.903387  


## Double Sorting by dividend yield and market equity

In [30]:
#Create 5 portfolios based on dv dataframe (companies with positive dividend yield)
print("Create portfolios")
t = time.time() # reset our timer

# loop over all years in the data
# Note: the first loop loops over the years in range(1981,2017).
#    You can wrap any list by the tqdm command to display a progress bar while looping over the list
portfolios_ds = [] # create an empty list to collect the portfolio returns
for year in tqdm(range(1985,2017),desc="years"):
    # take the companies that were alive at t-1
    permno_list=list(dv[dv['year']==year-1]['permno'].unique()) 
    
    # get the sorting variable for these companies at t-1
    sorting_data = dv.loc[(dv['year']==(year-1)) & \
                           (dv['permno'].isin(permno_list)), \
                           ['permno','permco','dvy_a','market_cap']]
    sorting_data=sorting_data.sort_values(['dvy_a','market_cap']).reset_index(drop=True)
    
    if(len(sorting_data)==0):
        continue
    
    # sort into 5 baskets by cashflow over assets
    nportfolios = 4 # number of portfolios
    #sorting_data['dvy_a','market_cap']
    sorting_data['rank'] = pd.qcut(sorting_data.index,nportfolios, labels=False)#, precision=4)
    
    # select the return data with some time lag to make sure that the accounting information is public (data from July at year t to June in year t+1)
    crsp_window = crsp[((crsp['year']==year) & (crsp['month']>=6)) | \
                       ((crsp['year']==year+1) & (crsp['month']<=6))]
    
    # create the portfolio returns for the current window and collect them in portfolios_window
    portfolios_window = [] 
    for p in range(nportfolios):
        # get list of permnos that are in this portfolio
        basket = sorting_data.loc[sorting_data['rank'] == p,'permno'].tolist()
        
        # get returns of these permnos
        crsp_p_firms = crsp_window[crsp_window['permno'].isin(basket)]
        
        # pivot returns
        returns = crsp_p_firms.pivot(index='date', columns='permno', values='ret')
        returns = returns.iloc[1:,:] # drop the first row
        
        # create equally weighted portfolio (monthly rebalancing)
        return_port = returns.mean(axis=1)
        return_port.name = str(p)
        
        # collect portfolio returns in dec_port
        portfolios_window += [return_port]
        
    # merge the portfolios
    portfolios_window = pd.concat(portfolios_window,axis=1)
        
    # collect results in portfolios
    portfolios_ds += [portfolios_window]

# merge the returns from all windows
portfolios_ds = pd.concat(portfolios_ds,axis=0)


print('Step 3 completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block



Create portfolios


years: 100%|███████████████████████████████████████████████████████████████████████████| 32/32 [00:02<00:00, 15.27it/s]

Step 3 completed in 2.1s


In [31]:
portfolios_ds['4']=portfolios_ds['3']-portfolios_ds['0']

In [32]:
#Create 5 portfolios based on dv dataframe (companies with positive dividend yield)
print("Create portfolios")
t = time.time() # reset our timer

# loop over all years in the data
# Note: the first loop loops over the years in range(1981,2017).
#    You can wrap any list by the tqdm command to display a progress bar while looping over the list
portfolios_ds_0 = [] # create an empty list to collect the portfolio returns
for year in tqdm(range(1985,2017),desc="years"):
    # take the companies that were alive at t-1
    permno_list=list(dv_0[dv_0['year']==year-1]['permno'].unique()) 
    
    # get the sorting variable for these companies at t-1
    sorting_data = dv.loc[(dv_0['year']==(year-1)) & \
                           (dv_0['permno'].isin(permno_list)), \
                           ['permno','permco','dvy_a','market_cap']]
    sorting_data=sorting_data.sort_values(['dvy_a','market_cap']).reset_index(drop=True)
    
    if(len(sorting_data)==0):
        continue
    
    # sort into 5 baskets by cashflow over assets
    nportfolios = 1 # number of portfolios
    #sorting_data['dvy_a','market_cap']
    sorting_data['rank'] = pd.qcut(sorting_data.index,nportfolios, labels=False)#, precision=4)
    
    # select the return data with some time lag to make sure that the accounting information is public (data from July at year t to June in year t+1)
    crsp_window = crsp[((crsp['year']==year) & (crsp['month']>=6)) | \
                       ((crsp['year']==year+1) & (crsp['month']<=6))]
    
    # create the portfolio returns for the current window and collect them in portfolios_window
    portfolios_window = [] 
    for p in range(nportfolios):
        # get list of permnos that are in this portfolio
        basket = sorting_data.loc[sorting_data['rank'] == p,'permno'].tolist()
        
        # get returns of these permnos
        crsp_p_firms = crsp_window[crsp_window['permno'].isin(basket)]
        
        # pivot returns
        returns = crsp_p_firms.pivot(index='date', columns='permno', values='ret')
        returns = returns.iloc[1:,:] # drop the first row
        
        # create equally weighted portfolio (monthly rebalancing)
        return_port = returns.mean(axis=1)
        return_port.name = str(p)
        
        # collect portfolio returns in dec_port
        portfolios_window += [return_port]
        
    # merge the portfolios
    portfolios_window = pd.concat(portfolios_window,axis=1)
        
    # collect results in portfolios
    portfolios_ds_0 += [portfolios_window]

# merge the returns from all windows
portfolios_ds_0 = pd.concat(portfolios_ds_0,axis=0)


print('Step 3 completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block



Create portfolios


years: 100%|███████████████████████████████████████████████████████████████████████████| 32/32 [00:01<00:00, 16.69it/s]

Step 3 completed in 1.9s


In [33]:
portfolios_ds['5'] =portfolios_ds_0['0']
portfolios_ds

,0,1,2,3,4,5
date,,,,,,
1987-07-31,0.038191,0.040972,0.042870,0.021221,-0.016970,0.037230
1987-08-31,0.029609,0.032789,0.035770,0.026213,-0.003396,0.030932
1987-09-30,-0.026768,-0.018685,-0.006135,-0.012750,0.014018,-0.016656
1987-10-30,-0.272076,-0.251540,-0.246018,-0.177002,0.095075,-0.240797
1987-11-30,-0.052665,-0.046193,-0.034254,-0.020470,0.032195,-0.039686
...,...,...,...,...,...,...
2010-02-26,0.047873,0.035243,0.007620,0.041849,-0.006024,0.038760
2010-03-31,0.070751,0.083484,0.088512,0.043454,-0.027298,0.095319
2010-04-30,0.055061,0.076558,0.120142,0.163848,0.108787,0.077118


### Step 4: Performance Evaluation For double Sorting
#### Step 4a: Merge Portfolio returns with Fama French data

In [34]:
portfolios_ds.to_excel('portfolios_doublesorted.xlsx')

### Step 4b: Regressions

In [35]:
# show average returns (annualized and in percent)
print("Average returns (annualized percent)\n",((1+portfolios_ds.mean(axis=0))**12-1)*100)

# Calculate the excess returns
for p in range(6):
    portfolios_ff['ExRet_'+str(p)] = portfolios_ff[str(p)]-portfolios_ff['RF']

Average returns (annualized percent)
 0    10.740896
1    11.544543
2    11.414925
3    12.492999
4     1.595771
5    12.638745
dtype: float64


In [36]:
### Market model regressions ###
table_capm = []
for p in range(6):
    # regress portfolio excess return on market excess return
    results = sm.OLS(portfolios_ff['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff['ExMkt'])).fit()
    
    # collect results
    table_row = pd.DataFrame({'alpha':results.params['const'],
                              'beta_mkt':results.params['ExMkt'],
                              'alpha_t':results.tvalues['const'],
                              'rmse':np.sqrt(results.mse_resid),
                              'R2':results.rsquared},
                             index=[p])
    
    table_capm += [table_row]

In [37]:
# Combine the results for all portfolios
table_capm = pd.concat(table_capm,axis=0)
table_capm.index.name = 'quintile'

# show results
print("CAPM\n",table_capm)

CAPM
              alpha  beta_mkt   alpha_t      rmse        R2
quintile                                                  
0         0.001520  0.888347  0.954061  0.022679  0.754634
1         0.002645  0.746606  1.475068  0.025525  0.631670
2         0.002749  0.691243  1.418261  0.027593  0.557119
3         0.003808  0.623045  1.673264  0.032396  0.425753
4        -0.001491 -0.264915 -0.845319  0.025106  0.182463
5         0.003359  0.776022  2.057416  0.023240  0.690883


In [38]:
### Three Factor model regressions ###
table_ff = []
for p in range(6):
    # regress portfolio excess return on market excess return
    results = sm.OLS(portfolios_ff['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff[['ExMkt','SMB','HML']])).fit()
    
    # collect results
    table_row = pd.DataFrame({'alpha':results.params['const'],
                              'beta_mkt':results.params['ExMkt'],
                              'beta_size':results.params['SMB'],
                              'beta_hml':results.params['HML'],
                              'alpha_t':results.tvalues['const'],
                              'rmse':np.sqrt(results.mse_resid),
                              'R2':results.rsquared},
                             index=[p])
    
    table_ff += [table_row]

# Combine the results for all portfolios
table_ff = pd.concat(table_ff,axis=0)
table_ff.index.name = 'quintile'

# show results
print("Fama-French 3\n",table_ff)

Fama-French 3
              alpha  beta_mkt  beta_size  beta_hml   alpha_t      rmse  \
quintile                                                                
0        -0.000790  0.979954   0.405876  0.599966 -0.782840  0.014218   
1        -0.000118  0.878141   0.366892  0.714892 -0.101198  0.016376   
2        -0.000363  0.842390   0.397482  0.805113 -0.305473  0.016757   
3         0.000421  0.776966   0.489441  0.877368  0.278483  0.021282   
4        -0.002608 -0.201787   0.094682  0.287830 -1.527846  0.024038   
5         0.000707  0.895368   0.389454  0.687045  0.762596  0.013057   

                R2  
quintile            
0         0.904521  
1         0.849895  
2         0.838284  
3         0.754630  
4         0.257959  
5         0.903387  
